In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import os
from google.colab import files

projectID = 'finalproject-ecmms'
location = 'US'
client = bigquery.Client(project=projectID, location=location)

auth.authenticate_user()

print('authenticated')

In [ ]:
queries = {
    '4-dc_categories': """
        SELECT
          distribution_center_id,
          category,
          COUNT(*) AS count
        FROM `bigquery-public-data.thelook_ecommerce.products`
        GROUP BY distribution_center_id, category
    """,
    '4-dc_position': """
        SELECT * FROM `bigquery-public-data.thelook_ecommerce.distribution_centers`
    """,
    '4-dc_sold_stock': """
        SELECT
          product_distribution_center_id,
          COUNT(CASE WHEN sold_at IS NULL THEN 1 END) AS non_disponibili,
          COUNT(CASE WHEN sold_at IS NOT NULL THEN 1 END) AS disponibili,
          COUNT(*) AS totale,
          ROUND(((COUNT(CASE WHEN sold_at IS NOT NULL THEN 1 END) / COUNT(*)) * 100), 1) AS percentuale_disponibili
        FROM `bigquery-public-data.thelook_ecommerce.inventory_items`
        GROUP BY product_distribution_center_id
        ORDER BY percentuale_disponibili ASC
    """,
    '4-dc_availabity': """
        SELECT
          product_id,
          product_distribution_center_id,
          COUNT(sold_at IS NULL) AS disponibilita
        FROM `bigquery-public-data.thelook_ecommerce.inventory_items`
        GROUP BY product_id, product_distribution_center_id
    """,
    '4-dc_revenue': """
        SELECT
          product_distribution_center_id,
          EXTRACT(YEAR FROM sold_at) AS year,
          EXTRACT(MONTH FROM sold_at) AS month,
          SUM((product_retail_price - cost)) AS monthly_revenue
        FROM `bigquery-public-data.thelook_ecommerce.inventory_items`
        WHERE sold_at IS NOT NULL
        GROUP BY product_distribution_center_id, month, year
        ORDER BY monthly_revenue ASC
    """
}

for query_name, query in queries.items():
    query_job = client.query(query)
    results = query_job.result().to_dataframe()
    filename = f'{query_name}.csv'
    results.to_csv(filename, index=False)
    print(f'Risultati "{query_name}" salvati come CSV')
    files.download(filename)

print('Download completato')
